# Demo: Control of wheel angle with partial state feedback

#### Import modules and configure the notebook.

In [ ]:
# These are standard modules
import time
import numpy as np
import sympy as sym
from scipy.signal import place_poles
from scipy import linalg
from scipy import signal
import matplotlib.pyplot as plt
from IPython.display import display, Latex, Markdown

# This is a custom interface to the pybullet simulator
import ae353_wheel_sensors

# Suppress the use of scientific notation when printing small numbers
np.set_printoptions(suppress=True)

#### Choose parameter values

The equation of motion is

$$c_1 \ddot{q} = \tau - c_2\sin q$$

where

* $q$ is the wheel angle
* $\tau$ is the wheel torque, i.e., the torque applied to the wheel by a motor on the ground

and where the values of $c_1$ and $c_2$ are given as follows:

In [ ]:
# Values of c1 and c2 for the example (do not change)
c1 = 1.0
c2 = 2.0

# Values of other parameters that are given
g = 9.81 # <-- acceleration of gravity
r = 0.35 # <-- distance from wheel axis to marker (i.e., to the mass)

# Values of other parameters that correspond to the coefficients c1 and c2 that we chose
m = c2 / (g * r)   # <-- set in code
J = c1 - m * r**2  # <-- set in URDF
print(f'm = {m}\nJ = {J}')

#### Set up simulations

Create an instance of the `Simulator` class with particular values of `roll`, `mass`, and `inertia`.

In [ ]:
simulator = ae353_wheel_sensors.Simulator(
    display=True,
    roll=(np.pi / 2),
    mass=m,
    inertia=J,
)

#### Define functions to plot results

This function plots results in terms of nonlinear states and inputs.

In [ ]:
def show_results_mno(data, q_e, v_e, tau_e):
    t = data['t']
    q = data['wheel_angle']
    v = data['wheel_velocity']
    tau = data['wheel_torque']
    tau_cmd = data['wheel_torque_command']
    q_meas = data['wheel_angle_measurement']
    xhat = data['xhat']
    qhat = xhat[:, 0] + q_e
    vhat = xhat[:, 1] + v_e

    fig, (ax_q, ax_v, ax_tau) = plt.subplots(
        3, 1, figsize=(10, 8), sharex=True,
    )

    ax_q.plot(t, q, label=r'$q$', linewidth=4)
    ax_q.plot(t, qhat, '--', label=r'$\widehat{q}$', linewidth=4)
    ax_q.plot(t, q_meas, '.', markersize=3, label=r'$q_\text{meas}$')
    ax_q.plot(t, q_e * np.ones_like(t), ':', label=r'$q_e$', linewidth=3, color='C3')
    ax_v.plot(t, v, label=r'$v$', linewidth=4)
    ax_v.plot(t, vhat, '--', label=r'$\widehat{v}$', linewidth=4)
    ax_v.plot(t, v_e * np.ones_like(t), ':', label=r'$v_e$', linewidth=3, color='C3')
    ax_tau.plot(t, tau, label=r'$\tau$', linewidth=4)
    ax_tau.plot(t, tau_cmd, '-.', label=r'$\tau$ (command)', linewidth=3, color='C6')
    tau_max = np.ones_like(t) * simulator.tau_max
    ax_tau.plot(t, tau_max,
                '--', label=r'$\tau$ (max)', linewidth=2, color='C2', zorder=0)
    ax_tau.plot(t, - tau_max,
                '--', linewidth=2, color='C2', zorder=0)

    ax_q.grid()
    ax_q.legend(fontsize=16, ncol=4, loc='upper right')
    ax_q.tick_params(labelsize=14)
    ax_q.set_ylim(q_e - 0.5, q_e + 0.5)

    ax_v.grid()
    ax_v.legend(fontsize=16, ncol=3, loc='upper right')
    ax_v.tick_params(labelsize=14)
    ax_v.set_ylim(v_e - 1., v_e + 1.)

    ax_tau.grid()
    ax_tau.legend(fontsize=16, ncol=3, loc='upper right')
    ax_tau.tick_params(labelsize=14)
    ax_tau.set_ylim(-1.2 * simulator.tau_max, 1.2 * simulator.tau_max)

    ax_tau.set_xlabel('time (s)', fontsize=20)
    ax_tau.set_xlim([data['t'][0], data['t'][-1]])

    fig.tight_layout()
    plt.show()

This function plots results in terms of linear states and inputs.

In [ ]:
def show_results_xuy(data, q_e, v_e, tau_e):
    t = data['t']
    q = data['wheel_angle']
    v = data['wheel_velocity']
    tau = data['wheel_torque']
    tau_cmd = data['wheel_torque_command']
    q_meas = data['wheel_angle_measurement']
    xhat = data['xhat']
    x1 = q - q_e
    x2 = v - v_e
    u1 = tau - tau_e
    u1_cmd = tau_cmd - tau_e
    y1 = q_meas - q_e
    
    fig, (ax_x1, ax_x2, ax_u1) = plt.subplots(
        3, 1, figsize=(10, 8), sharex=True,
    )

    ax_x1.plot(t, x1, label=r'$x_1$', linewidth=4)
    ax_x1.plot(t, xhat[:, 0], '--', label=r'$\widehat{x}_1$', linewidth=4)
    ax_x1.plot(t, y1, '.', markersize=3, label=r'$y_1$')
    ax_x2.plot(t, x2, label=r'$x_2$', linewidth=4)
    ax_x2.plot(t, xhat[:, 1], '--', label=r'$\widehat{x}_2$', linewidth=4)
    ax_u1.plot(t, u1, label=r'$u_1$', linewidth=4)
    ax_u1.plot(t, u1_cmd, '-.', label=r'$u_1$ (command)', linewidth=3, color='C6')
    tau_max = np.ones_like(t) * simulator.tau_max
    ax_u1.plot(t, tau_max - tau_e,
               '--', label=r'$u_1$ (max)', linewidth=2, color='C2', zorder=0)
    ax_u1.plot(t, - tau_max - tau_e,
               '--', linewidth=2, color='C2', zorder=0)
            
    ax_x1.grid()
    ax_x1.legend(fontsize=16, ncol=3, loc='upper right')
    ax_x1.tick_params(labelsize=14)
    ax_x1.set_ylim(-0.5, 0.5)
        
    ax_x2.grid()
    ax_x2.legend(fontsize=16, ncol=2, loc='upper right')
    ax_x2.tick_params(labelsize=14)
    ax_x2.set_ylim(-1., 1.)
    
    ax_u1.grid()
    ax_u1.legend(fontsize=16, ncol=3, loc='upper right')
    ax_u1.tick_params(labelsize=14)
    ax_u1.set_ylim(tau_e + -1.2 * simulator.tau_max, tau_e + 1.2 * simulator.tau_max)
    
    ax_u1.set_xlabel('time (s)', fontsize=20)
    ax_u1.set_xlim([data['t'][0], data['t'][-1]])
    
    fig.tight_layout()
    plt.show()

This function plots results in terms of *both* nonlinear and linear states and inputs.

In [ ]:
def show_results(data, q_e, v_e, tau_e):
    t = data['t']
    q = data['wheel_angle']
    v = data['wheel_velocity']
    tau = data['wheel_torque']
    tau_cmd = data['wheel_torque_command']
    q_meas = data['wheel_angle_measurement']
    xhat = data['xhat']
    qhat = xhat[:, 0] + q_e
    vhat = xhat[:, 1] + v_e
    x1 = q - q_e
    x2 = v - v_e
    u1 = tau - tau_e
    u1_cmd = tau_cmd - tau_e
    y1 = q_meas - q_e

    fig, ((ax_q, ax_x1), (ax_v, ax_x2), (ax_tau, ax_u1)) = plt.subplots(
        3, 2, figsize=(15, 10), sharex=True,
    )

    ax_q.plot(t, q, label=r'$q$', linewidth=4)
    ax_q.plot(t, qhat, '--', label=r'$\widehat{q}$', linewidth=4)
    ax_q.plot(t, q_meas, '.', markersize=3, label=r'$q_\text{meas}$')
    ax_q.plot(t, q_e * np.ones_like(t), ':', label=r'$q_e$', linewidth=3, color='C3')
    ax_v.plot(t, v, label=r'$v$', linewidth=4)
    ax_v.plot(t, vhat, '--', label=r'$\widehat{v}$', linewidth=4)
    ax_v.plot(t, v_e * np.ones_like(t), ':', label=r'$v_e$', linewidth=3, color='C3')
    ax_tau.plot(t, tau, label=r'$\tau$', linewidth=4)
    ax_tau.plot(t, tau_cmd, '-.', label=r'$\tau$ (command)', linewidth=3, color='C6')
    tau_max = np.ones_like(t) * simulator.tau_max
    ax_tau.plot(t, tau_max,
                '--', label=r'$\tau$ (max)', linewidth=2, color='C2', zorder=0)
    ax_tau.plot(t, - tau_max,
                '--', linewidth=2, color='C2', zorder=0)

    ax_q.grid()
    ax_q.legend(fontsize=16, ncol=4, loc='upper right')
    ax_q.tick_params(labelsize=14)
    ax_q.set_ylim(q_e - 0.5, q_e + 0.5)

    ax_v.grid()
    ax_v.legend(fontsize=16, ncol=3, loc='upper right')
    ax_v.tick_params(labelsize=14)
    ax_v.set_ylim(v_e - 1., v_e + 1.)

    ax_tau.grid()
    ax_tau.legend(fontsize=16, ncol=3, loc='upper right')
    ax_tau.tick_params(labelsize=14)
    ax_tau.set_ylim(-1.2 * simulator.tau_max, 1.2 * simulator.tau_max)

    ax_tau.set_xlabel('time (s)', fontsize=20)
    ax_tau.set_xlim([data['t'][0], data['t'][-1]])

    ax_x1.plot(t, x1, label=r'$x_1$', linewidth=4)
    ax_x1.plot(t, xhat[:, 0], '--', label=r'$\widehat{x}_1$', linewidth=4)
    ax_x1.plot(t, y1, '.', markersize=3, label=r'$y_1$')
    ax_x2.plot(t, x2, label=r'$x_2$', linewidth=4)
    ax_x2.plot(t, xhat[:, 1], '--', label=r'$\widehat{x}_2$', linewidth=4)
    ax_u1.plot(t, u1, label=r'$u_1$', linewidth=4)
    ax_u1.plot(t, u1_cmd, '-.', label=r'$u_1$ (command)', linewidth=3, color='C6')
    tau_max = np.ones_like(t) * simulator.tau_max
    ax_u1.plot(t, tau_max - tau_e,
               '--', label=r'$u_1$ (max)', linewidth=2, color='C2', zorder=0)
    ax_u1.plot(t, - tau_max - tau_e,
               '--', linewidth=2, color='C2', zorder=0)
            
    ax_x1.grid()
    ax_x1.legend(fontsize=16, ncol=3, loc='upper right')
    ax_x1.tick_params(labelsize=14)
    ax_x1.set_ylim(-0.5, 0.5)
        
    ax_x2.grid()
    ax_x2.legend(fontsize=16, ncol=2, loc='upper right')
    ax_x2.tick_params(labelsize=14)
    ax_x2.set_ylim(-1., 1.)
    
    ax_u1.grid()
    ax_u1.legend(fontsize=16, ncol=3, loc='upper right')
    ax_u1.tick_params(labelsize=14)
    ax_u1.set_ylim(tau_e + -1.2 * simulator.tau_max, tau_e + 1.2 * simulator.tau_max)
    
    ax_u1.set_xlabel('time (s)', fontsize=20)
    ax_u1.set_xlim([data['t'][0], data['t'][-1]])

    fig.tight_layout()
    plt.show()

This function plots errors.

In [ ]:
def show_results_err(data, q_e, v_e, tau_e):
    t = data['t']
    q = data['wheel_angle']
    v = data['wheel_velocity']
    xhat = data['xhat']
    x1 = q - q_e
    x2 = v - v_e
    x1hat = xhat[:, 0]
    x2hat = xhat[:, 1]
    
    fig, ((ax_x1, ax_xerr1), (ax_x2, ax_xerr2)) = plt.subplots(
        2, 2, figsize=(15, 5), sharex=True,
    )

    ax_x1.plot(t, x1, label=r'$x_1 - 0$', linewidth=4)
    ax_x2.plot(t, x2, label=r'$x_2 - 0$', linewidth=4)

    ax_xerr1.plot(t, x1hat - x1, label=r'$\widehat{x}_1 - x_1$', linewidth=4)
    ax_xerr2.plot(t, x2hat - x2, label=r'$\widehat{x}_2 - x_2$', linewidth=4)

    ax_xerr1.grid()
    ax_xerr1.legend(fontsize=16, loc='upper right')
    ax_xerr1.tick_params(labelsize=14)
    ax_xerr1.set_ylim(-0.5, 0.5)

    ax_xerr2.grid()
    ax_xerr2.legend(fontsize=16, loc='upper right')
    ax_xerr2.tick_params(labelsize=14)
    ax_xerr2.set_ylim(-1., 1.)
            
    ax_x1.grid()
    ax_x1.legend(fontsize=16, loc='upper right')
    ax_x1.tick_params(labelsize=14)
    ax_x1.set_ylim(-0.5, 0.5)
        
    ax_x2.grid()
    ax_x2.legend(fontsize=16, loc='upper right')
    ax_x2.tick_params(labelsize=14)
    ax_x2.set_ylim(-1., 1.)
    
    ax_x2.set_xlabel('time (s)', fontsize=20)
    ax_x2.set_xlim([data['t'][0], data['t'][-1]])

    ax_xerr2.set_xlabel('time (s)', fontsize=20)
    ax_xerr2.set_xlim([data['t'][0], data['t'][-1]])

    ax_x1.set_title('ERROR IN THE STATE', fontsize=20, fontweight='bold')
    ax_xerr1.set_title('ERROR IN THE STATE ESTIMATE', fontsize=20, fontweight='bold')

    fig.tight_layout()
    plt.show()

#### Do controller design

Derive state-space model.

In [ ]:
# Define symbolic variables
q, v, tau = sym.symbols('q, v, tau')

# Define ODEs
f = sym.Matrix([
    v,
    (tau - c2 * sym.sin(q)) / c1,
])

# Convert floats to rationals
f = sym.nsimplify(f, rational=True)

# Choose equilibrium point
(q_e, v_e, tau_e) = (np.pi, 0., 0.)

# Linearize
A_num = sym.lambdify([q, v, tau], f.jacobian([q, v]))
A = A_num(q_e, v_e, tau_e).astype(float)
B_num = sym.lambdify([q, v, tau], f.jacobian([tau]))
B = B_num(q_e, v_e, tau_e).astype(float)

# Show result
print(f'A =\n{A}\n\nB =\n{B}')

Verify the system is controllable.

In [ ]:
W_c = np.block([B, A @ B])
print(f'       rank(W_c) = {np.linalg.matrix_rank(W_c)}')
print(f'number of states = {A.shape[0]}')

Design controller by eigenvalue placement.

In [ ]:
p_c = [-2., -5.]
print(f'p_c =\n{p_c}\n')

K = signal.place_poles(A, B, p_c).gain_matrix
print(f'K =\n{K}')

#### Do observer design

Derive state-space model. In particular, given a nonlinear model

$$o = g\left(\begin{bmatrix} q \\ v \end{bmatrix}, \begin{bmatrix} \tau \end{bmatrix}\right) = \begin{bmatrix} q \end{bmatrix}$$

of sensor measurements, linearize the function $g$ about the equilibrium point $q_e, v_e, \tau_e$ to produce a state-space model

$$y = Cx + Du$$

where

$$
\begin{aligned}
y
&= o - g\left(\begin{bmatrix} q \\ v \end{bmatrix}, \begin{bmatrix} \tau \end{bmatrix}\right) \\
&= \begin{bmatrix} q \end{bmatrix} - \begin{bmatrix} q_e \end{bmatrix} \\
&= \begin{bmatrix} q - q_e \end{bmatrix}
\end{aligned}
$$

and

$$
C = \frac{\partial g}{\partial \begin{bmatrix} q \\ v \end{bmatrix}}\Biggr\rvert_{\left(\begin{bmatrix} q_e \\ v_e \end{bmatrix}, \begin{bmatrix} \tau_e \end{bmatrix}\right)}
\qquad\qquad
D = \frac{\partial g}{\partial \begin{bmatrix} \tau \end{bmatrix}}\Biggr\rvert_{\left(\begin{bmatrix} q_e \\ v_e \end{bmatrix}, \begin{bmatrix} \tau_e \end{bmatrix}\right)}.
$$

In [ ]:
# Define nonlinear model
g = sym.Matrix([q])

# Convert floats to rationals
g = sym.nsimplify(g, rational=True)

# Linearize
C_num = sym.lambdify([q, v, tau], g.jacobian([q, v]))
C = C_num(q_e, v_e, tau_e).astype(float)
D_num = sym.lambdify([q, v, tau], g.jacobian([tau]))
D = D_num(q_e, v_e, tau_e).astype(float)

# Show result
print(f'C =\n{C}\n\nD =\n{D}')

Verify the system is observable.

In [ ]:
W_o = np.block([[C], [C @ A]])
print(f'       rank(W_o) = {np.linalg.matrix_rank(W_o)}')
print(f'number of states = {A.shape[0]}')

Design observer by eigenvalue placement.

In [ ]:
p_o = [-4., -6.]
print(f'p_o =\n{p_o}\n')

L = signal.place_poles(A.T, C.T, p_o).gain_matrix.T
print(f'L =\n{L}')

#### Verify stability of controller, observer, and full system

Find eigenvalues of $A - BK$.

In [ ]:
print(linalg.eigvals(A - B @ K))

Find eigenvalues of $A - LC$.

In [ ]:
print(linalg.eigvals(A - L @ C))

Find eigenvalues of

$$\begin{bmatrix} A - BK & -BK \\ 0 & A - LC \end{bmatrix}.$$

In [ ]:
print(linalg.eigvals(np.block([
    [       A - B @ K,   - B @ K],
    [np.zeros_like(A), A - L @ C],
])))

Did we have to find the eigenvalues of this last big matrix? Or would it have sufficed only to find the eigenvalues of $A-BK$ and $A-LC$?

#### Implement and test

Implement.

In [ ]:
class Controller_WithObserver:
    def __init__(self, A, B, C, K, L, q_e, v_e, tau_e):
        self.A = A
        self.B = B
        self.C = C
        self.K = K
        self.L = L
        self.q_e = q_e
        self.v_e = v_e
        self.tau_e = tau_e
        
        self.dt = 0.01
        
        self.variables_to_log = ['xhat']
    
    def reset(self):
        self.xhat = np.array([
            0.,
            0.,
        ])
    
    def run(self, t, wheel_angle_measurement):
        """
        INPUTS
         t = current time (s)
         wheel_angle_measurement = measured angle of wheel from sensors (rad)
         
        OUTPUTS
         wheel_torque = torque applied to wheel by motor on ground (N m)
        """
        
        # Get input
        u = - self.K @ self.xhat
        
        # Get measured wheel angle (sensors)
        q = wheel_angle_measurement
        
        # Get output
        y = np.array([
            q - self.q_e,
        ])
        
        # Get state estimate
        self.xhat += self.dt * (
            self.A @ self.xhat + self.B @ u - self.L @ (self.C @ self.xhat - y)
        )
        
        # Get torque (actuators)
        wheel_torque = u[0] + self.tau_e
        
        return wheel_torque

Test.

In [ ]:
# Create controller
controller = Controller_WithObserver(A, B, C, K, L, q_e, v_e, tau_e)

# Choose initial conditions
(q_i, v_i) = (q_e + 0.2, v_e)

# Reset controller
controller.reset()

# Reset simulator
simulator.reset(
    wheel_angle=q_i,
    wheel_velocity=v_i,
    sensor_noise=0.,        # <-- CHANGE (e.g., to 0.1) AND SEE WHAT HAPPENS
)

# Reset controller
controller.reset()

# Run simulator (with display)
simulator.display_meshcat = True
data = simulator.run(
    controller,
    max_time=5.0,
)

# Show results
show_results(data, q_e, v_e, tau_e)

Distinguish between "error in the state" and "error in the state estimate."

In [ ]:
show_results_err(data, q_e, v_e, tau_e)

What happens if you slow down the controller?

In [ ]:
# Design controller
p_c = [-2., -5.]
# p_c = [-0.2, -0.5]
K = signal.place_poles(A, B, p_c).gain_matrix

# Design observer
p_o = [-4., -6.]
L = signal.place_poles(A.T, C.T, p_o).gain_matrix.T

# Create controller
controller = Controller_WithObserver(A, B, C, K, L, q_e, v_e, tau_e)

# Choose initial conditions
(q_i, v_i) = (q_e + 0.2, v_e)

# Reset controller
controller.reset()

# Reset simulator
simulator.reset(
    wheel_angle=q_i,
    wheel_velocity=v_i,
    sensor_noise=0.,        # <-- CHANGE (e.g., to 0.1) AND SEE WHAT HAPPENS
)

# Reset controller
controller.reset()

# Run simulator (without display)
simulator.display_meshcat = False
data = simulator.run(
    controller,
    max_time=5.0,
)

# Show results
show_results_err(data, q_e, v_e, tau_e)

What happens if you slow down the observer?

In [ ]:
# Design controller
p_c = [-2., -5.]
K = signal.place_poles(A, B, p_c).gain_matrix

# Design observer
p_o = [-4., -6.]
# p_o = [-0.4, -0.6]
L = signal.place_poles(A.T, C.T, p_o).gain_matrix.T

# Create controller
controller = Controller_WithObserver(A, B, C, K, L, q_e, v_e, tau_e)

# Choose initial conditions
(q_i, v_i) = (q_e + 0.2, v_e)

# Reset controller
controller.reset()

# Reset simulator
simulator.reset(
    wheel_angle=q_i,
    wheel_velocity=v_i,
    sensor_noise=0.,        # <-- CHANGE (e.g., to 0.1) AND SEE WHAT HAPPENS
)

# Reset controller
controller.reset()

# Run simulator (without display)
simulator.display_meshcat = False
data = simulator.run(
    controller,
    max_time=5.0,
)

# Show results
show_results_err(data, q_e, v_e, tau_e)

#### Try optimal controller and observer design

Define a function to solve the continuous-time infinite-horizon LQR problem.

In [ ]:
def lqr(A, B, Q, R):
    P = linalg.solve_continuous_are(A, B, Q, R)
    K = linalg.inv(R) @  B.T @ P
    return K

Design an optimal controller:

* $Q_c$ must be a square $n_x \times n_x$ matrix where $n_x$ is the length of $x$
* $R_c$ must be a square $n_u \times n_u$ matrix where $n_u$ is the length of $u$

In [ ]:
Q_c = np.eye(2)
R_c = np.eye(1)
K = lqr(A, B, Q_c, R_c)

print(f'K =\n{K}')
print(f'\np_c:\n{linalg.eigvals(A - B @ K)}')

Design an optimal observer:

* $Q_o$ must be a square $n_y \times n_y$ matrix where $n_y$ is the length of $y$
* $R_o$ must be a square $n_x \times n_x$ matrix where $n_x$ is the length of $x$

In [ ]:
Q_o = np.eye(1)
R_o = np.eye(2)
L = lqr(A.T, C.T, np.linalg.inv(R_o), np.linalg.inv(Q_o)).T

print(f'L =\n{L}')
print(f'\np_o:\n{linalg.eigvals(A - L @ C)}')

Test.

In [ ]:
# Create controller
controller = Controller_WithObserver(A, B, C, K, L, q_e, v_e, tau_e)

# Choose initial conditions
(q_i, v_i) = (q_e + 0.2, v_e)

# Reset controller
controller.reset()

# Reset simulator
simulator.reset(
    wheel_angle=q_i,
    wheel_velocity=v_i,
    sensor_noise=0.,        # <-- CHANGE (e.g., to 0.1) AND SEE WHAT HAPPENS
)

# Reset controller
controller.reset()

# Run simulator (with display)
simulator.display_meshcat = True
data = simulator.run(
    controller,
    max_time=5.0,
)

# Show results
show_results(data, q_e, v_e, tau_e)

Show errors.

In [ ]:
show_results_err(data, q_e, v_e, tau_e)